In [96]:
import numpy as np
import tqdm.notebook as tqdm
import random
import pandas as pd

# Generate initial data

In [97]:
CASTLES = 13
SOLDIERS = 100
ENTRIES = 12000
ENTRIES2 = 3000
ENTRIES3 = 1000
SQUARE_WEIGHT = True

In [98]:
def get_entry():
    alloc = np.zeros(CASTLES, int)
    for i in range(SOLDIERS):
        c = random.randint(0,CASTLES-1)
        alloc[c] += 1
    return alloc

In [99]:
subs = []

for n in tqdm.trange(ENTRIES):
    subs.append(get_entry())

# Build scoring function

In [100]:
score_subs = subs

def score(alloc):
    values = np.array(range(1,CASTLES+1))
    w = t = l = 0
    for irow in score_subs:
        delta = np.subtract(alloc, irow)
        signs = np.sign(delta)
        s = np.dot(signs, values)
        if s > 0: w += 1
        elif s < 0: l += 1
        else: t += 1
#     print(w,t,l)
    return w + t/2

In [101]:
%%time
all_scores = []
for alloc in tqdm.tqdm(subs):
    all_scores.append(score(alloc))
max_score = max(all_scores)
idx_max = all_scores.index(max_score)
print(subs[idx_max], max_score)


[ 4  4  8  2  6  4 11  3  7 12 11 15 13] 11615.5
CPU times: user 5min 32s, sys: 14.9 s, total: 5min 47s
Wall time: 5min 36s


Now create a smaller list of entries based on how successful they were.

In [102]:
subs2 = random.choices(subs, k=ENTRIES2, weights = np.array(all_scores)**2 if SQUARE_WEIGHT else all_scores)
score_subs = subs2

Do that again.

In [103]:
%%time
all_scores = []
for alloc in tqdm.tqdm(subs2):
    all_scores.append(score(alloc))
max_score = max(all_scores)
idx_max = all_scores.index(max_score)
print(subs2[idx_max], max_score)


[ 4  4  8  2  6  4 11  3  7 12 11 15 13] 2895.5
CPU times: user 21.4 s, sys: 1.56 s, total: 23 s
Wall time: 21.7 s


In [104]:
subs3 = random.choices(subs2, k=ENTRIES3, weights = np.array(all_scores)**2 if SQUARE_WEIGHT else all_scores)
score_subs = subs3

# Iteratively, look for better options

In [105]:
def get_tweak(alloc):
    s = score(alloc)
    deltas = [0] * CASTLES
    for i in range(CASTLES):
        new_alloc = alloc.copy()
        new_alloc[i] += 1
        new_s = score(new_alloc)
        deltas[i] = new_s-s
    inc = deltas.index(max(deltas))
    deltas = [0] * CASTLES
    for i in range(CASTLES):
        new_alloc = alloc.copy()
        if new_alloc[i] > 0:
            new_alloc[i] -= 1
            new_s = score(new_alloc)
            deltas[i] = new_s-s
        else:
            deltas[i] = -55
    dec = deltas.index(max(deltas))
    return((inc, dec))

In [106]:
def optimize(a):
    alloc = a.copy()
    s = 0
    new_s = score(alloc)
    
    while new_s > s:
        s = new_s
#         print(f"{alloc} yields {s}")

        inc, dec = get_tweak(alloc)

        if inc==dec: break

        alloc[inc]+=1
        alloc[dec]-=1

        new_s = score(alloc)

    return alloc

In [107]:
subs_optimized = []

for alloc in tqdm.tqdm(subs3):
    new_alloc = optimize(alloc.copy())
    subs_optimized.append(new_alloc)

In [132]:
subs4 = subs3 + subs_optimized
score_subs = subs4

In [134]:
%%time
all_scores = list(map(score, subs_optimized))
max_score = max(all_scores)
idx_max = all_scores.index(max_score)
print(f"{subs_optimized[idx_max]} --> {max_score} / {2*ENTRIES3}, params = {ENTRIES} / {ENTRIES2} / {ENTRIES3} / {SQUARE_WEIGHT}")

[ 0  6  0 10 12 13  0 13 15 14  0 17  0] --> 1996.0 / 2000, params = 12000 / 3000 / 1000 / True
CPU times: user 4.39 s, sys: 9.61 ms, total: 4.4 s
Wall time: 4.4 s


In [135]:
df = pd.DataFrame(subs_optimized)
df['score'] = all_scores

In [136]:
df.sort_values('score', ascending=False).head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,score
640,0,6,0,10,12,13,0,13,15,14,0,17,0,1996.0
998,1,0,8,0,0,12,11,13,14,5,15,17,4,1993.0
454,1,0,8,0,0,12,11,13,14,5,15,17,4,1993.0
808,0,0,8,10,12,11,13,0,14,0,15,0,17,1989.5
93,1,0,8,9,6,13,3,13,14,0,0,16,17,1989.5
434,0,5,0,9,11,11,12,4,14,0,0,17,17,1988.0
735,1,6,7,0,2,12,12,13,14,0,15,17,1,1987.5
879,1,6,5,1,9,12,5,0,14,14,0,16,17,1986.5
234,0,4,0,10,6,6,0,12,14,14,15,16,3,1985.0
442,0,4,0,10,6,6,0,12,14,14,15,16,3,1985.0


In [138]:
final = df.sort_values('score', ascending=False).iloc[0].values.astype(int)[:-1]
final

array([ 0,  6,  0, 10, 12, 13,  0, 13, 15, 14,  0, 17,  0])

In [139]:
final = [ 0,  6,  0, 10, 12, 13,  0, 13, 15, 14,  0, 17,  0]
print(sum(final))
score(final)

100


1996.0

# Results

Stored results of different runs

[ 1  3  9  0  9  6 12  0  8 12 12 14 14] --> 750.0 / 750, params = 10000 / 3000 / 750 / True

[ 1  3  3  0 11  9  7 13 12 12 16 13  0] --> 750.0 / 750, params = 10000 / 3000 / 750 / False